<a href="https://colab.research.google.com/github/srulikbd/Deep-Learning/blob/master/Train_MLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install datasets

     |████████████████████████████████| 778kB 4.2MB/s 
     |████████████████████████████████| 890kB 8.5MB/s 
     |████████████████████████████████| 3.0MB 20.3MB/s 
     |████████████████████████████████| 1.1MB 55.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=24faa68d6ff1853d0d3aafdb90ed27793a525ac8a4eb964986ca483d3646a3c4
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from transformers import LongformerForMaskedLM, Trainer, TrainingArguments, LongformerTokenizer
from datasets import load_dataset

model = LongformerForMaskedLM.from_pretrained("allenai/longformer-large-4096")
tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-large-4096")


Some weights of LongformerForMaskedLM were not initialized from the model checkpoint at allenai/longformer-large-4096 and are newly initialized: ['lm_head.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total # of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
)




In [ ]:
def load_data(path):
  files_path = sorted(os.listdir(path))
  dataset = load_dataset('text', data_files={'train': files_path})
  return dataset

In [ ]:
%%time
from transformers import LineByLineTextDataset

# dataset = LineByLineTextDataset(
#     tokenizer=tokenizer,
#     file_path="/content/sample_data/patent.txt",
#     block_size=128,
# )
train_dataset = load_data(path)
test_dataset = train_dataset[1000:]
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)


trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    data_collator=data_collator,
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset            # evaluation dataset
)

CPU times: user 3.78 s, sys: 1.86 s, total: 5.65 s
Wall time: 14.3 s


In [ ]:
trainer.train()

TrainOutput(global_step=3, training_loss=2.456737518310547)